In [1]:
%display latex
%display default

## Précision numérique

Par défaut, les nombres flottants et les calculs flottants sont fait avec une précision de 53 bits

In [2]:
a= 3.0
a.parent()

Real Field with 53 bits of precision

In [3]:
a.precision()

53

In [4]:
(1.0/3.0).parent()

Real Field with 53 bits of precision

In [5]:
print(1/3, (1/3).parent())
print( (1/3).n(), (1/3).n().parent())
print(RR(1/3), RR(1/3).parent())


1/3 Rational Field
0.333333333333333 Real Field with 53 bits of precision
0.333333333333333 Real Field with 53 bits of precision


Les nombres flottants sont définis comme instance de l'objet `RealField`. On peut spécifier la précision de cette objet.

In [6]:
R=RealField()
R10=RealField(10)
R200=RealField(200)

R.precision(), R10.precision(), R200.precision()

(53, 10, 200)

In [7]:
a   = R(3.513)
a10 = R10(3.513)
a200= R200(3.513)

a^100, a10^100, a200^100

(3.69670656868857e54,
 3.6e54,
 3.6967065686885807935368708162449489665828918404074416864066e54)

Calcul de $0.3-0.2-0.1$ avec différentes précisions

In [8]:
a,b,c=.3,.2,.1
a10,b10,c10=R10(.3),R10(.2),R10(.1)
a200,b200,c200=R200(.3),R200(.2),R200(.1)
a-b-c,a10-b10-c10,a200-b200-c200

(-2.77555756156289e-17,
 -0.00012,
 7.7787690973264271339300800672251553007378152109014589163764e-62)

**Attention:** 
- si on ne précise pas RealField, la précision est calculée automatiquement.
- si les calculs sont mixtes (opérandes avec différentes précisions), les calculs sont fait dans la plus petite précision

In [9]:
a=0.5555555555554033333333323232323233232111117
a.precision(), a.parent()

(143, Real Field with 143 bits of precision)

In [10]:
a10-b200-c200, a10-b10-c10

(-0.00012, -0.00012)

In [11]:
a= 0.1e-20
b= 0.1
c= 0.3e-10
a+b+c-b,b-b+a+c,a+c+b-b

(3.00000024822111e-11, 3.00000000010000e-11, 3.00000024822111e-11)

**A RETENIR**
> - `RealField(p)` est l'objet définissant les approximations flottantes à précision $p$
> - `numerical_approx(expr, p)` ou `n(expr,p)` ou `expr.n(p)` retourne l'approximation flottante de l'expression `expr` à précision `p`
> - par défaut `RealField(53)` est utilisé (déjà défini dans `RR`)

## Exemple de problème de précision

### calcul de racine carré

In [12]:
a=51
b=a^61
print("l'entier b est codée sur "+str( b.nbits()) + " bits")
r=sqrt(b)
r

l'entier b est codée sur 347 bits


1686961934066707040236155036109474174986501789839001*sqrt(51)

In [13]:
s   = R(sqrt(b))
s200= R200(sqrt(b))
print(floor(s),floor(s200))
floor(s)^2 <= b <(floor(s)+1)^2,\
floor(s200)^2 <= b <=(floor(s200)+1)^2

12047317913813611992216795507023424262776490557440000 12047317913813610588023392961569665056411324497659166


(False, True)

## résolution de système d'équations linéaires 


In [14]:
N=2
MS=MatrixSpace(QQ,N)
VS=VectorSpace(QQ,N)
A=MS.random_element(density=1,num_bound=10,den_bound=10)
b=VS.random_element(num_bound=100,den_bound=100)
V=var(list("x"+ str(i) for i in range(N)))
X=vector(V)
eq=list((A*X)[i]==b[i] for i in range(N))

### La fonction `solve`

elle prend une liste d'équations symboliques et une liste d'inconnues

In [15]:
print("variables:",V)
print("Equations:",eq)
#for i in range(N):
#    print(eq[i])

variables: (x0, x1)
Equations: [6*x0 - 10*x1 == (3/19), 4*x0 + x1 == (88/29)]


In [16]:
sol=solve(eq, V)
sol

[[x0 == (16807/25346), x1 == (4842/12673)]]

Est-il possible de faire la même chose mais de faire le calcul numériquement ?

In [17]:
eqf=list((A.n()*X)[i]==b[i].n() for i in range(N))
print("variables:",V)
print("Equations:",eqf)
#for i in range(N):
#    print(eqf[i])

variables: (x0, x1)
Equations: [6.00000000000000*x0 - 10.0000000000000*x1 == 0.157894736842105, 4.00000000000000*x0 + 1.00000000000000*x1 == 3.03448275862069]


In [18]:
solf=solve(eqf,V)
solf

[[x0 == (16807/25346), x1 == (4842/12673)]]

Quoi, le résultat est le même !!!

En fait, `solve` fait appel à une méthode de résolution symbolique. 
Cela implique que les réels sont transformées en nombre rationnel lors de la résolution. 

### l'algèbre linéaire

In [19]:
show(eq)

[6*x0 - 10*x1 == (3/19), 4*x0 + x1 == (88/29)]

In [20]:
show(A, (V), b)

[  6 -10]
[  4   1] (x0, x1) (3/19, 88/29)

In [21]:
y=A.solve_right(b)
show(y)

(16807/25346, 4842/12673)

Pareil mais en numérique

In [22]:
Af=A.numerical_approx()
bf=b.n()
show(Af,V,bf)

[ 6.00000000000000 -10.0000000000000]
[ 4.00000000000000  1.00000000000000] (x0, x1) (0.157894736842105, 3.03448275862069)

In [23]:
yf=Af.solve_right(bf)
show(yf)

(0.663102659196717, 0.382072121833820)

In [24]:
# difference des solutions
y.n()-yf

(0.000000000000000, 0.000000000000000)

Tout pareil mais en plus grand

In [25]:
N=80
MS=MatrixSpace(QQ,N)
VS=VectorSpace(QQ,N)
A=MS.random_element(density=1,num_bound=100,den_bound=100)
b=VS.random_element(num_bound=100,den_bound=100)


In [26]:
#print(A,b)
%time y=A.solve_right(b)

CPU times: user 41 s, sys: 453 ms, total: 41.5 s
Wall time: 43 s


In [27]:
y[0]

5797033771855442240799692097146947321090327501125463385766057598801810756512243781109800194217043548308664081235591484643429002811089690038532547338013203255830811974754996213707514427072203264450134762075794298366410896014174048448139937400973240232353007535448196312728317298748399908303392315734420644967814482515414390951387928628634204076756261672323388504844189589781535983821826710568220855202266749860884143863107407353106102003155022684010448130931906272980698919279909579272447406202257166518643226665462058084179760156555187239114132118368223634813370948781990769883042303028689149365655699301575646507254113004131005047225720631936320827404166378392739127451705274424758180876019688822785700708551083991354733182884240350357004500714858010340091600464445006434929441986714416962235542120374171140831564221218136378299095371353552431722330468308461324951353085355252547047719417032009093500687739505999396444915416146891160075464585300416074502835066231704699528792231740907809819493589945

In [28]:
print("num(y[0])=",y[0].numerator(),'\n')
print("den(y[0])=",y[0].denominator())

num(y[0])= 57970337718554422407996920971469473210903275011254633857660575988018107565122437811098001942170435483086640812355914846434290028110896900385325473380132032558308119747549962137075144270722032644501347620757942983664108960141740484481399374009732402323530075354481963127283172987483999083033923157344206449678144825154143909513879286286342040767562616723233885048441895897815359838218267105682208552022667498608841438631074073531061020031550226840104481309319062729806989192799095792724474062022571665186432266654620580841797601565551872391141321183682236348133709487819907698830423030286891493656556993015756465072541130041310050472257206319363208274041663783927391274517052744247581808760196888227857007085510839913547331828842403503570045007148580103400916004644450064349294419867144169622355421203741711408315642212181363782990953713535524317223304683084613249513530853552525470477194170320090935006877395059993964449154161468911600754645853004160745028350662317046995287922317409078098

In [29]:
Af=A.n()
bf=b.n()
%time yf= Af.solve_right(bf)

CPU times: user 1.06 s, sys: 0 ns, total: 1.06 s
Wall time: 1.09 s


In [30]:
print("yf[0]=",yf[0])
#erreur commise par la résolution numérique
y[0]-yf[0], (Af*yf)[0]- bf[0]

yf[0]= 5.17390261885681


(1.24535937118253e-10, -2.08821848701746e-12)

In [31]:
N=20
A=matrix(N,N,lambda i,j : 1/(i+j+1))
b=vector([ZZ.random_element() for i in range(N)])
show(A)
show(b)

20 x 20 dense matrix over Rational Field (use the '.str()' method to see the entries)

(-1, -1, -1, -1, 2, -1, 0, -1, 0, 0, 0, 3, 0, 10, -1, 13, -1, 11, 1, 3)

In [32]:
Af=A.n()
bf=b.n()
%time y = A.solve_right(b)
%time yf= Af.solve_right(bf)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.93 ms
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 28.6 ms


In [33]:
### le résultat avec les flottants est complètement faux
print("yf[0]=",yf[0])
(Af*yf)[0], bf[0]

yf[0]= 5.87774940485942e10


(2.57443200000000e6, -1.00000000000000)

In [34]:
### le résultat avec les rationnels est exact
print("y[0]=",y[0])
(A*y)[0],b[0],A*y==b

y[0]= -10426097185622140


(-1, -1, True)

**A RETENIR**
- la fonction `solve` est une méthode de résolution symbolique
- les calculs avec les flottants peuvent être plus rapides mais aussi faux (surtout en algèbre linéaire)
- les calculs avec les rationnels entrainent un grossissement des données mais le résultat est toujours correct

## Limite du calcul symbolique

In [35]:
var('a b c')
e1=a*c+b*c
e2=(a+b)*c
e1.show()
e2.show()
(e2 == e1.collect(c)).show()
(e1 == e2.expand()).show()
bool(e1==e2)

a*c + b*c

(a + b)*c

(a + b)*c == (a + b)*c

a*c + b*c == a*c + b*c

True

In [36]:
e=(a-b)^2-a^2+2*a*b-b^2
e.show()
bool(e==0)

(a - b)^2 - a^2 + 2*a*b - b^2

True

In [37]:
var('x')
e= cos(x)^2+sin(x)^2
e, e.simplify_trig()

(cos(x)^2 + sin(x)^2, 1)

In [38]:
e=sqrt(x^2)
bool(x==e), bool(x==e.simplify_full()),bool(x==e.canonicalize_radical())

(False, False, True)

### Explosion combinatoire des symboles

In [39]:
var('x')
A=Matrix(2,2,[[cos(x),sin(x)],[-sin(x),cos(x)]])
show("A=",A,"  det(A)=",A.determinant())

'A=' [ cos(x)  sin(x)]
[-sin(x)  cos(x)] '  det(A)=' cos(x)^2 + sin(x)^2

In [40]:
N=100
d=(A^N).determinant()
print("d=",d)

#A^N,(A^(N)).expand()

d= 16*(16*(256*(16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (64*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2)^2)*(128*(64*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2)*(16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 + (256*(16*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*sin(x)^2 - (4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2)^2*(4*cos(x)^2*sin(x)^2 - (cos(x)^2 - sin(x)^2)^2)^2*(cos(x)^2 - sin(x)^2)^2*cos(x)^2*si

In [41]:
%time print(bool(d==1))
%time print(bool(d.simplify_trig()==1))

True
CPU times: user 9.44 s, sys: 141 ms, total: 9.58 s
Wall time: 9.89 s
True
CPU times: user 2.14 s, sys: 0 ns, total: 2.14 s
Wall time: 2.01 s


### Pas toujours de solutions

In [42]:
var('x')
(x^2+1).roots()

[(-I, 1), (I, 1)]

In [43]:
E= (x+1)^5 - (x+2)^4
F= (x+1)^5 + (x+2)^4
EF=E*F
print(f"{E=}\n{F=}\n{EF=}")
EF.roots(x)

SyntaxError: invalid syntax (<fstring>, line 1)

In [ ]:
E= (x+1)^10 - (x+2)^9
F= (x+1)^10 + (x+2)^9
EF=E*F
print(f"{E=}\n{F=}\n{EF=}")
EF.roots(x) ## aucunes solution trouvées alors que c'est un polynôme

Pour résoudre on passe du calcul symbolique au calcul algébrique.

Pour cela on se place dans la structure mathématique des polynomes qui sont des expressions symboliques ayant de bonne propriétés: en particulier une représentation canonique !!!

Un polynôme est une expression de la forme : $a_0+a_1X+\dots+a_nX^n$ avec $n$ une valeur connue et les coefficients $\{a_0,a_1,\dots,a_n\}$ qui sont tous du même type (Entier, Rationnel, Complexe, ...).

$\Rightarrow$ le vecteur $[a_0,a_1,\dots,a_n]$ est une représentation canonique de ce polynôme.

In [ ]:
# Ensemble des polynomes à coefficient dans QQ
# on choisit la variable X comme symbole des polynômes
PR.<X>=PolynomialRing(QQ) #ou PR.<X>=QQ[]

print("type de x: ",x.parent())
print("type de X: ",X.parent())
bool(X==x)

In [ ]:
print((2*X+4).parent()) 
print((2*x+4).parent()) 
print((2*X+ cos(3)).parent())
print((2*X+0.5).parent()) # attention coercion vers les polynomes à coefficient dans RR
RR

In [ ]:
# on convertit les expressions symboliques vers les polynomes en substituant 
# la variable x par celle des polynômes en X et en construisant le polynôme associé
polyE=PR(E.subs(x=X)) 
polyF=PR(F.subs(x=X))
polyEF=polyE*polyF
show(f"une expression symbolique: {E=}")
show(f"un polynomes: {polyE=}")


In [ ]:
# calcul des racines dans le domaine de définition du polynome (ici Q)
print(f"calcul des racines sur {polyEF.base_ring()}")
polyEF.roots()

In [ ]:
# calcul des racines dans C
R=polyEF.roots(CC)
R

In [ ]:
R[0][0],polyEF(R[0][0])

In [ ]:
polyEF(ComplexField(80)(R[0][0]))

In [ ]:
# calcul de la forme canonique d'un produit de polynômes
E1= (x+1)^100 - (x+2)^99
F1= (x+1)^100 + (x+2)^99
polyE1=PR(E.subs(x=X)) 
polyF1=PR(F.subs(x=X))

# via la representation par arbre (exp symbolique)
%time (E1*F1).expand() 
# via la representation par liste des coefficients
%time polyEF1=polyE1*polyF1 

**A RETENIR**
- le calcul symbolique permet de manipuler n'importe quelles expressions faisant intervenir des symboles et des fonctions 
- pas toujours possible de trouver des solutions avec uniquement des calculs symboliques
- l'utilisation de calcul algèbrique est souvent préférable si on connaît la structure mathématique sous-jacente (Ex. les polynômes, les matrices, ...)
- le calcul algébrique sera la plupart du temps plus rapide
- parfois on retrouve un mélange de calcul algébrique et numérique (résolution dans les complexes si le problème est défini sur les nombres rationnels)

## Les matrices

In [ ]:
A=matrix([[1,2],[3,4]])
B=matrix([[1,2,3],[4,5,6]])
print(A,B)
show(A,B)

In [ ]:
# addition de matrice
show(A+A)
# multiplication par un scalaire
show(3*A , A+A+A)

In [ ]:
show("A=",A)
show("B=",B)

A+B

### matrices et combinaisons linéaires

In [ ]:
# une combinaison linéaire de symboles
var('x y z')
e=[x,y,z]
3*e[0]+2*e[1]+5*e[2]

In [ ]:
u=matrix(1,3,[1,2,3])
v=matrix(3,1,e)
show(u,v)

In [ ]:
# le produit de matrice n*1 par 1*n est une combinaison linéaire
u*v

In [ ]:
# on peut augmenter le nombre de combinaisons linéaires (on ajoute une ligne à gauche)
B=matrix(2,3,[1,2,3,4,5,6])
show(B,v)
show(B*v)

In [ ]:
# on peut augmenter le nombre de combinaisons linéaires (on ajoute une colonne à droite)
var('x,y,z,a,b,c')
V=matrix(3,2,[[x,a],[y,b],[z,c]])
show(V)
show(B*V)

In [ ]:
# ATTENTION: les tailles des matrices doivent correspondre
show(V)
V*V

In [ ]:
# ATTENTION: le produit matriciel n'est pas commutatif
bool(V*B==B*V)

In [ ]:
show(V*B,B*V)

In [ ]:
# On peut transposer une matrice câd intervertir A[i,j] avec A[j,i]
show(V)
show(V.transpose())

In [ ]:
# le produit d'une matrice par sa transposée est toujours bien défini
show(V.transpose()*V)
show(V*V.transpose())

### Les espaces de matrices

In [ ]:
# le premier paramètres est le domaine des coefficients (optionnel)
# si il est présent il impose le type des coefficients
A=matrix([[1,2],[3,4]])
B=matrix(RR,[[1,2/3],[3,4]])
C=matrix([[1,2/3],[3,4]])
A.parent(),B.parent(),C.parent()

In [ ]:
matrix(ZZ,[[1,2/3],[3,4]])

In [ ]:
# le second/troisièmes paramètres sont les dimensions (optionnel)
# si ils sont présents ils imposent le nombre de coefficients (si aucun que des zéros)
A=matrix(ZZ,2,2)
B=matrix(ZZ,2,2,[[4,5],[6,7]])
C=matrix(ZZ,2,2,[4,5,6,7])
show(A)
show(B)
show(C)

In [ ]:
# Comme les matrices vivent dans un espace de matrice on peut définir cet objet
MS23=MatrixSpace(ZZ,2,3)
MS22=MatrixSpace(QQ,2,2)
show(MS22,"\tet\t",MS23)
MS22,MS23

In [ ]:
MS23.dimension() , MS23.dims()

In [ ]:
A=MS22.matrix([1,2,3,4])
A.parent()

In [ ]:
MS22.random_element()
show(_)

In [ ]:
MS10=MatrixSpace(ZZ,10,10)
MS10.random_element(density=1,x=0,y=100)
show(_)

In [ ]:
MS10=MS10.change_ring(QQ)
MS10.random_element(density=0.5,num_bound=100,den_bound=100)
show(_)

### un peu de calcul algébrique matriciel

In [ ]:
N=10
q=17
MS=MatrixSpace(GF(q),N,N)
A=MS.random_element()
show("A=",A)
show(f"det(A)={A.det()}")

In [ ]:
B=A.change_ring(ZZ)
show("B=",B)
show(f"det(B)={B.det()}")

In [ ]:
B.det()%q == A.det()

In [ ]:
show(A^-1)

In [ ]:
show(B^-1)

In [ ]:
(B^(-1)%q), A^-1
show(_)

In [ ]:
# Expliquer le résultat obtenu
(B^(-1)%q) ==  A^-1

In [ ]:
(B^(-1)%q).parent(),(A^-1).parent()

In [ ]:
(B^(-1)%q) ==  (A^-1).change_ring(B.base_ring())

In [ ]:
(B^(-1)).change_ring(A.base_ring()) ==  (A^-1)